# Instructions

Write SQL queries for each of the following questions. Be sure to save your answers often. When you are done, clear the results from the notebook using the "eraser" icon in the top right corner of the ADS window. Then save once more, and upload your Notebook to Moodle.

In all cells where you see `use [stu.dent]`, replace *stu.dent* with your user name.

## Part 1 - Single Table Queries

**Q1.** In what year was **Jennifer Lawrence** born?  (1990)

In [ ]:
use [jack.rutherford]
select BirthYear from imdb.People
where FirstName = 'Jennifer' and LastName = 'Lawrence'

**Q2.** In the `SchoolSchedulingExample` schema, list the `SubjectName` for all `Subjects` that have a subject code **starting with** _MUS_ (4 rows)

In [ ]:
use [SchoolSchedulingExample]
select SubjectName from Subjects
where SubjectCode like 'MUS%'

**Q3.**  In the `SchoolSchedulingExample` schema, show the `SubjectID`, `StartTime` and `Duration` for each `Class` that meest for less than 2 hours on either **Monday/Wednesday/Friday** or **Tuesday/Thursday**.  

The `MondaySchedule`, `TuesdaySchedule`, `WednesdaySchedule`, `ThursdaySchedule`, `FridaySchedule`, and `SaturdaySchedule` columns in the `Classes` table have either 0 or 1 in them, depending on whether the class in question meets (1) or does not meet (0).  

The output should return a column named `Meets` whose value is either **MWF** or **TR**, depending on when the class meets.  Use the SQL Server [CASE statement](https://link.hope.edu/392-sql-case) to compute the value of the `Meets` column.

Show the classes that meet Monday/Wednesday/Friday **first**, followed by the Tuesday/Thursday classes. Within the MWF and TR groups, order the classes by `StartTime`.  (14 rows)

In [ ]:
use [SchoolSchedulingExample];
with ClassDays as(
select SubjectID, StartTime, Duration,
CASE
    WHEN MondaySchedule = 1 AND TuesdaySchedule = 0 AND ThursdaySchedule = 0 AND WednesdaySchedule = 1 AND FridaySchedule = 1 AND SaturdaySchedule = 0 THEN 'MWF'
    WHEN MondaySchedule = 0 AND TuesdaySchedule = 1 AND ThursdaySchedule = 1 AND WednesdaySchedule = 0 AND FridaySchedule = 0 AND SaturdaySchedule = 0 THEN 'TR'
    END AS Meets
from Classes
where Duration < 120
)
select * from ClassDays where Meets is not NULL
    order by Meets, StartTime

**Q4.**  What was the title of the first movie **Jennifer Lawrence** appeared in, and what year was it released?  Use the `ActorMovies` view and the [SQL Server TOP clause](https:\link.hope.edu\392-sql-top) to help you answer this question.  (_The Poker House, 2008_)

In [ ]:
use [jack.rutherford]
select Top 1 Title, YearReleased from imdb.Movies m
JOIN imdb.Principals pr on m.TitleID = pr.TitleID
join imdb.People pe on pe.PersonID = pr.PersonID
where FirstName = 'Jennifer' and LastName = 'Lawrence'
order by YearReleased

**Q5.**  In the `BowlingLeagueExample` schema, show the game numbers where a bowler bowled either a raw score over 192, or a handicapped score over 224, but still lost the game.  The values in the `WonGame` column or either **0 (lost)** or **1 (won)**. (2 rows)

In [ ]:
use BowlingLeagueExample
select GameNumber from Bowlers b
join Bowler_Scores bs on b.BowlerID = bs.BowlerID
where (RawScore > 192 OR HandiCapScore > 224) AND WonGame = 0

**Q6.**  List the title, year released, and IMDB rating of all the 'G' rated movies released in 2000-2009 for which the primary genre was either _Animation_ or _Family_ and the IMDB rating for the movie was at least 6.  Order the results so the highest rated movies are shown first.  <span style="color: var(--vscode-foreground);">(13 rows, the first is *Cars* and the last is *Franklin and the Turtle Lake Treasure*)</span>

In [ ]:
use [jack.rutherford]
select Title, YearReleased, IMDBRating from imdb.Movies m
where YearReleased between 2000 and 2009
and MPAARating = 'G'
and (PrimaryGenre = 'Animation' or PrimaryGenre = 'Family')
and IMDBRating >=6
order by IMDBRating desc

## Part 2 - Multiple Table Queries

**Q7.**  Repeat the above question, except find movies for which **any** of the genres associated with the movie were either _Animation_ or _Family._ The other criteria from the previous problem should be kept, and each movie should appear once in the results (57 rows, the first is _The Nutcracker Family: Behind the Magic_ and the last is _Jimmy Neutron: Boy Genius_).

In [ ]:
use [jack.rutherford]
select Title, YearReleased, IMDBRating from imdb.Movies m
where YearReleased between 2000 and 2009
and MPAARating = 'G'
and IMDBRating >=6
and m.TitleID in(
    select mg.TitleID from imdb.MovieGenres mg
    where mg.Genre = 'Animation' 
    or mg.Genre = 'Family'
)
order by IMDBRating desc

**Q8.** Find the names and birth years of all people with last names starting with A through C who were born in the 1980s and whose primary profession is _composer_.

Order by results by year of birth; within a year, sort alphabetically by last name, followed by first name (47 rows; the first is _Francisco Aguilar_ born in 1980 and the last is _Dexter Britain_ born in 1989)

In [ ]:
use [jack.rutherford]
SELECT FirstName, LastName, BirthYear FROM imdb.People
join imdb.Professions on Professions.PersonID = People.PersonID
where Professions.Profession = 'composer'
and Professions.ProfessionOrder =1
and BirthYear between 1980 and 1989
and LastName like '[a-c]%'
group by BirthYear, LastName, FirstName
order by LastName asc, FirstName asc

**Q9.**  List the team name and the name of the captain for each of the bowling teams _Marlins_, _Sharks_, and _Terrapins_ (3 rows).

In [ ]:
use [BowlingLeagueExample]
select TeamName, BowlerFirstName, BowlerLastName from Teams
join Bowlers on Bowlers.TeamID = Teams.TeamID
where CaptainID = BowlerID
and BowlerID in(
    select BowlerID from Bowlers
    where TeamName = 'Marlins' or TeamName = 'Sharks' or TeamName = 'Terrapins'
)


**Q10.**  Show the product name, vendor name, and number on hand for all vendors who supply products containing the word _Bike_ which are running low. Running low means there are fewer than 10 of the product on hand.

Show the products closest to being out of stock first. If there is a tie, show the product that will take the fewest days to deliver first <span style="color: var(--vscode-foreground);">(8 rows; the first is</span> _Viscount Mountain Bike_ <span style="color: var(--vscode-foreground);">supplied by</span> _Viscount_<span style="color: var(--vscode-foreground);">, and the last is</span> _Eagle FS-3 Mountain Bike_ <span style="color: var(--vscode-foreground);">supplied by</span> _Big Sky Mountain Bikes_<span style="color: var(--vscode-foreground);">).</span>

In [ ]:
use [SalesOrdersExample]
select ProductName, VendName, QuantityOnHand from Products
join Product_Vendors on Product_Vendors.ProductNumber = Products.ProductNumber
join Vendors on Vendors.VendorID = Product_Vendors.VendorID
where QuantityOnHand < 10
and ProductName like '%Bike%'
order by QuantityOnHand asc, DaysToDeliver asc

##  Handing it in

Click the "eraser" icon in the top right corner of the ADS window to clear the results from all cells in your notebook.  Then go to Moodle and upload the notebook using the **Homework 2** assignment in the "Assignments" section of the site.